# The purpose of this file is to run our machine learning model and upload the data to Google FireBase. This file should be set to run daily, so new predictions can be uploaded to the Google FireBase. These predicted prices (plus the model RMSE) will be pulled from the database to be displayed.

In [2]:
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from datetime import datetime, timedelta
from firebase import firebase

#Getting our data
for stock in ['AAPL', 'NFLX', 'AMZN', 'TSLA', 'FB', 'GOOG', 'MSFT']:
    today = datetime.now()
    today = today.strftime("%Y-%m-%d")

    df = web.DataReader(stock, data_source = 'yahoo', start = '2012-01-01', end = today)

    #Creating a new dataframe with only the close column
    data = df[['Close']]

    #Converting dataframe to numpy array
    dataset = data.values

    #Get the number of rows to train the model on (we want to train on 80% of the data)
    training_data_len = math.ceil(len(dataset) * 0.8)

    #Normalizing the data - we always do this whenever we input data into machine learning models (they perform better)
    scaler = MinMaxScaler(feature_range = (0, 1))
    scaled_data = scaler.fit_transform(dataset)

    #Create the scaled training dataset
    train_data = scaled_data[0: training_data_len, :]

    #Split the data into x_train (independent training variables) and y_train (dependent training variables - target variable) data sets
    x_train = []
    y_train = []

    #Here, we're basically taking every 60 "timepoints" of data, then appending the next value after that to y_train.
    #The purpose is to use 60 data points to predict the 61st, over and over again, to train our model.
    #0 - 59 --> we append the first y_train value (60), then 1 --> 60, we append the next y_train value (61)
    for i in range(60, len(train_data)):
        x_train.append(train_data[i - 60: i, 0])
        y_train.append(train_data[i, 0])

    #Convert the x_train and y_train to numpy arrays (proper format to use in the model)
    x_train, y_train = np.array(x_train), np.array(y_train)

    #Reshape the data because the model expects a 3d input: number of samples, numebr of timepoints, and number of features
    #Right now, our data is still 2-D

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    #Build the LSTEM Model - the model is constructed in "layers"
    model = Sequential()
    model.add(LSTM(50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
    model.add(LSTM(50, return_sequences = False))
    model.add(Dense(25))
    model.add(Dense(1))

    #Compile the model
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')

    #Train the model (fit is another word for train)
    model.fit(x_train, y_train, batch_size = 1, epochs = 1)

    #Creating the normalized testing dataset
    test_data = scaled_data[training_data_len - 60: , :]

    #Creating x_test (Values we want the model to use to predict y_test) and y_test (values we want our model to predict)
    x_test = []
    y_test = dataset[training_data_len: , :]

    for i in range(60, len(test_data)):
        x_test.append(test_data[i - 60: i, 0])

    #Converting the data to a numpy array
    x_test = np.array(x_test)

    #Reshape the data to make it 3-D (because that's what the model is expecting)
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    #Get the models predicted price values, and un-normalizing them
    predictions = model.predict(x_test)
    predictions = scaler.inverse_transform(predictions) 

    #Evaluating our models performance - by getting the Root Mean Square Error (RMSE)
    #Lower values of RMSE means it's a better fit
    #A value of 0 mean that the predictions are perfect
    rmse = np.sqrt(np.mean(((predictions - y_test)**2)))
    print('Root Mean Squared Value: ')
    print(rmse)

    #Let's try to make a prediction
    #Getting the data
    apple_quote2 = web.DataReader(stock, data_source = 'yahoo', start = '2012-01-01', end = today)

    #Create a new dataframe
    new_df = apple_quote2[['Close']]

    #Get the last 60 days of closing price values and convert the dataframe to an array
    last_60_days = new_df[-60:].values

    #Normalizing the data
    last_60_days_normalized = scaler.transform(last_60_days)

    #Create an empty list and appending the last 60 days worth of data
    X_test = []
    X_test.append(last_60_days_normalized)

    #Convert the X_test dataset to a numpy array
    X_test = np.array(X_test)

    #Reshaping the data
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    #Getting the predicted, normalized price
    predicted_price = model.predict(X_test)

    #Undo the normalization to get the actual price
    predicted_price = scaler.inverse_transform(predicted_price)
    print("Predicted Price is: " + str(predicted_price[0][0]))


    #Opens my database (no security)
    FBConn = firebase.FirebaseApplication('https://stock-price-predictor-9786f.firebaseio.com/', None)

    #Uploading data without an ID specification
    result = FBConn.put('/' + stock, 'RMSE', round(float(rmse), 2))
    result = FBConn.put('/' + stock, 'Predicted Price', round(float(predicted_price[0][0]), 2))
    result = FBConn.put('/' + stock, 'Date', today)
    
    print(stock + ' complete!')

Epoch 1/1
1658/1658 [==============================] - 57s 34ms/step - loss: 4.3057e-04
Root Mean Squared Value: 
10.087747498132366
Predicted Price is: 375.42892
AAPL complete!
Epoch 1/1
1658/1658 [==============================] - 59s 36ms/step - loss: 8.5752e-04
Root Mean Squared Value: 
16.63076897893863
Predicted Price is: 522.7446
NFLX complete!
Epoch 1/1
1658/1658 [==============================] - 58s 35ms/step - loss: 4.7737e-04
Root Mean Squared Value: 
90.69196332511449
Predicted Price is: 3102.5134
AMZN complete!
Epoch 1/1
1658/1658 [==============================] - 57s 34ms/step - loss: 1.9652e-04
Root Mean Squared Value: 
58.22239556087326
Predicted Price is: 1341.7794
TSLA complete!
Epoch 1/1
1582/1582 [==============================] - 57s 36ms/step - loss: 0.0013
Root Mean Squared Value: 
7.785316037293873
Predicted Price is: 232.3465
FB complete!
Epoch 1/1
1658/1658 [==============================] - 54s 32ms/step - loss: 9.6356e-04
Root Mean Squared Value: 
47.51818